In [110]:
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
from IPython.display import display, clear_output


In [ ]:
# Set up data
data = gpd.read_file('country_shapes/World_Countries__Generalized_.shp')
regions = pd.read_csv('country_shapes/regions.csv')

r = []
for c in data["COUNTRY"]:
    cr = regions[regions["name"] == c]["region"].values
    if len(cr) == 1:
        r.append(cr[0])
    else:
        r.append("None")
    
data["REGION"] = r

data.to_file("countries.shp")

In [140]:
'''
To do: 
    Region Selector
    Deploy
'''

# Read data
data = gpd.read_file("countries.shp")

# Create Interactives
countryText = widgets.Combobox(
    options=list(data["COUNTRY"]),
    description='Enter Country Name:',
    ensure_option=True,
    disabled=False
)
tryAgain = widgets.Button(description="Try Again")
showAnswer = widgets.Button(description="Show Answer")

nc = widgets.IntSlider(
    value=1,
    min=0,
    max=249,
    step=1,
    description='Number of Countries to Remove',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

# Remove secret country
# secrets = []
def generate_map(n_countries=1):
    clear_output()
    if isinstance(n_countries, dict):
        n_countries = n_countries.new["value"]
    secrets_idx = []
    global secrets
    secrets = []
    for n in range(n_countries):
        secret_idx = np.random.randint(len(data))
        secret = data.loc[secret_idx]
        
        secrets_idx.append(secret_idx)
        secrets.append(secret["COUNTRY"])

    display(data.drop(secrets_idx).explore(tiles=None, tooltip=False))
    return secrets
    
def guessFunc(guess):
    return guess

def ncFunc(n):
    print()
    secrets = generate_map(n)
    global n_countries
    n_countries = n
    return n

def tryAgainFunc(n):
    secrets = generate_map()
    showMap()

def showAnswerFunc(n):
    print("The remaining secret countr(y/ies) is/are: " + ", ".join(secrets))


def showMap():
    clear_output()
    display(tryAgain)
    display(showAnswer)
    interact(guessFunc, guess=countryText)
    interact(ncFunc, n=nc, names=['value'])
    


def box_enter(guess):
    if isinstance(guess.new, str):
        guess = guess.new
        try:
            if guess in secrets:
                secrets.remove(guess)
                print(guess + " is correct! There are " + str(len(secrets)) + " countries remaining")
                if len(secrets) == 0:
                    print("Congratulations! You found all the hidden countries!")
            else:
                print(guess + " is not correct, try again")
        except Exception as E:
            print(E)
#             pass
    
showMap()
countryText.observe(box_enter)
tryAgain.on_click(tryAgainFunc)
showAnswer.on_click(showAnswerFunc)

Button(description='Try Again', style=ButtonStyle())

Button(description='Show Answer', style=ButtonStyle())

interactive(children=(Combobox(value='', description='Enter Country Name:', ensure_option=True, options=('Amer…

interactive(children=(IntSlider(value=1, continuous_update=False, description='Number of Countries to Remove',…

In [139]:
print(secrets)

['Morocco']


In [117]:
help(widgets.Widget.observe)

Help on function observe in module traitlets.traitlets:

observe(self, handler, names=traitlets.All, type='change')
    Setup a handler to be called when a trait changes.
    
    This is used to setup dynamic notifications of trait changes.
    
    Parameters
    ----------
    handler : callable
        A callable that is called when a trait changes. Its
        signature should be ``handler(change)``, where ``change`` is a
        dictionary. The change dictionary at least holds a 'type' key.
        * ``type``: the type of notification.
        Other keys may be passed depending on the value of 'type'. In the
        case where type is 'change', we also have the following keys:
        * ``owner`` : the HasTraits instance
        * ``old`` : the old value of the modified trait attribute
        * ``new`` : the new value of the modified trait attribute
        * ``name`` : the name of the modified trait attribute.
    names : list, str, All
        If names is All, the handler will